In [ ]:
# raw data is incomplete. only biological data r1=~75-76 r2=~73-76
# breat cancer

In [1]:
import os
from parse_soft import ParseSoft
from search_soft import SearchSoft
from enrich_soft import EnrichSoft
from label_data import LabelData

%load_ext autoreload
%autoreload 2

In [3]:
# step 1: set defaults
geo = "GSE168410"
soft_dir = '/home/yuan/data'
meta_dir = '../data'
download_dir = '/home/yuan/rawdata/SRR'

In [4]:
# step 2: parse soft, and enrich

# analyze soft file
parser = ParseSoft(soft_dir)
soft_path = parser.soft_local_path(geo)
print('Retrieve meta data from soft file: ', soft_path)
softer = SearchSoft(soft_path)
data = softer.filter_data(softer.parse_rows)

# save to json
LabelData(meta_dir).save(data)

Retrieve meta data from soft file:  /home/yuan/data/ftp.ncbi.nlm.nih.gov/geo/series/GSE168nnn/GSE168410/soft/GSE168410_family.soft.gz
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


In [5]:
# step 3a: enrich data
data = LabelData(meta_dir).get_meta(geo)
enriched = EnrichSoft(data)()
LabelData(meta_dir).save(enriched)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


In [9]:
# step 3b: label samples manually
from label_sample import LabelSample

def collator(sample):
    ch = sample['characteristics']
    c = LabelSample(sample)
    c._protocol('scrna-seq', "10x3v2")
    c._disease_patient('breast cancer')
    c._update('tissue', 'breast, tumor')
    c._update('cell_type', ch['molecular_subtype'])
    if ch['neoadjuvant_treatment'] == 'yes':
        c._update('treatment', 'neoadjuvant')

data = LabelData(meta_dir).get_meta(geo)
for sample_id, sample in data['samples'].items():
    collator(sample)
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


In [11]:
for sample_id, sample in data['samples'].items():
    # print(sample['characteristics'])
    print(sample['sample_title'], sample['labels'])

Patient1P1L1 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'triple negative', 'treatment': 'neoadjuvant'}
Patient1P1L2 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'triple negative', 'treatment': 'neoadjuvant'}
Patient1P1L3 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'triple negative', 'treatment': 'neoadjuvant'}
Patient1P1L4 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'triple negative', 'treatment': 'neoadjuvant'}
Patient1P2L1 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'triple negative', 'treatment': 'neoadjuvant'}
Patient1P2L2 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'triple negative', 'treatment': 'neoadjuvant'}
Patient1P2L3 {'disease': 'breast cancer', 'group': 'patient', 'tissue': 'breast, tumor', 'cell_type': 'tri

In [12]:
# step 4a: load some data
from utils import Utils

# data are determined previously
samn_srr = Utils.from_json('../results/samn_srr.json')
print(len(samn_srr))

47


In [13]:
# step 4b: Given BioSample, parse SRR accessions into samples
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr(data, samn_srr)
LabelData(meta_dir).save(enriched)

biosamples = 47, SRR accessions = 47.
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


In [14]:
# step 5: merge srr_urls into data

# sample.<sample_id>.SRR.<SRR accession>."ftp.sra.ebi.ac.uk"
file_name = 'srr_fastq_urls_simple.json'
urls_json = os.path.join('../results', file_name)
urls = Utils.from_json(urls_json)

# parse urls
data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr_urls(data, urls)
LabelData(meta_dir).save(enriched)

{'srr': 47, 'available': 0, 'unknown': 47, 'updated': 0}
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


## download fastq.gz

In [15]:
# step 6a: download raw data using biosample accession using nextflow
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_biosample(sample_iter)
# run the bash script showed here
# Outcome: nf download failed

Number of biosamples:  47
Number of biosamples with SRR:  47
bash /home/yuan/rawdata/SRR/GSE168410/fetch1.sh


In [40]:
# step 6b: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
data, unparsing = ParseSra.parse_local_fastq(data, download_dir)
if unparsing:
    print('unparsed data: ', geo, len(unparsing), unparsing[:6])
LabelData(meta_dir).save(data)

unparsed data:  GSE168410 2 ['SRR13873714', 'SRR13873720']
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


In [41]:
#Step 7a: collect all SRR without local_fastq then execute bash run2.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_ebi_srr(sample_iter)
# run the bash script showed here

Number of SRR:  2
cd /home/yuan/rawdata/SRR/GSE168410 && bash fetch2.sh


In [43]:
# step 7b: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
data, unparsing = ParseSra.parse_local_fastq(data, download_dir)
if unparsing:
    print('unparsed data: ', geo, len(unparsing), unparsing[:6])
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE168nnn/GSE168410/metadata.json


In [44]:
#Step 8a: collect all SRR without local_fastq then execute bash run3.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_srr(sample_iter)
# run the bash script showed here

Number of SRR:  0
cd /home/yuan/rawdata/SRR/GSE168410 && bash fetch3.sh


## nf-core/scrnaseq pipeline
create samplehseet.csv and params.config

In [45]:
name = 'scrnaseq_breast_tissue_patient'
output_dir = '/home/yuan/output'
labeler = LabelData(meta_dir, name, output_dir, True)

sample_sheet = {}
data = LabelData(meta_dir).get_meta(geo)
for sample, geo, run_acc, fastq_sample in labeler.fastq_iter(data):
    cl = LabelSample(sample)
    print(sample['sample_id'], cl.filter(name))
    if cl.filter(name):
        keys = [cl.disease, geo, run_acc,]
        Utils.key_update(sample_sheet, [geo,], fastq_sample['sample_sheet'])
# export
if sample_sheet:
    print("Number of samples in samplesheet.csv: ", len(sample_sheet[geo]))
    labeler.to_sample_sheet(sample_sheet, 'paired')
    labeler.nf_cmd()
else:
    print(f'No samples meet creteria {name}')

GSM5138670 True
GSM5138671 True
GSM5138672 True
GSM5138673 True
GSM5138674 True
GSM5138675 True
GSM5138676 True
GSM5138677 True
GSM5138678 True
GSM5138679 True
GSM5138680 True
GSM5138681 True
GSM5138682 True
GSM5138683 True
GSM5138684 True
GSM5138685 True
GSM5138686 True
GSM5138687 True
GSM5138688 True
GSM5138689 True
GSM5138690 True
GSM5138691 True
GSM5138692 True
GSM5138693 True
GSM5138694 True
GSM5138695 True
GSM5138696 True
GSM5138697 True
GSM5138698 True
GSM5138699 True
GSM5138700 True
GSM5138701 True
GSM5138702 True
GSM5138703 True
GSM5138704 True
GSM5138705 True
GSM5138706 True
GSM5138707 True
GSM5138708 True
GSM5138709 True
GSM5138710 True
GSM5138711 True
GSM5138712 True
GSM5138713 True
GSM5138714 True
GSM5138715 True
GSM5138716 True
Number of samples in samplesheet.csv:  47
cd /home/yuan/output/scrnaseq_breast_tissue_patient/paired_GSE168410 && nextflow run nf-core/scrnaseq -r 3.0.0 -c params.config
